# GPT Practice

In [59]:
!pip3 install torch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [60]:
# Open data
f = open("input.txt", "r", encoding="utf-8")
text = f.read()
print(type(text))
text_length = len(text)
half_text_length = len(text) // 2

<class 'str'>


In [61]:
# Unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size

64

In [86]:
# Dictionary
char_to_int = {element: index for index, element in enumerate(chars)}
int_to_char = {index: element for index, element in enumerate(chars)}


# Encode str into int list
def encode(s):
    result = []
    for character in s:
        result.append(char_to_int[character])
    return result


# Decode int list into str
def decode(tensor):
    l = tensor.tolist()
    converted = []
    for int in l:
        converted.append(int_to_char[int])
    result = "".join(converted)
    return result

In [87]:
# Tensor
import torch
data = torch.tensor(data=encode(text), dtype=torch.long)

In [89]:
training_data = data[:half_text_length]
validation_data = data[half_text_length:]
validation_data
decode(training_data)

'KPop Demon Hunters is a 2025 American animated musical urban fantasy film. It was directed by Maggie Kang and Chris Appelhans, who also co-wrote the screenplay with Danya Jimenez and Hannah McMechan.\nThe story was originally created by Kang.\nThe film was produced by Sony Pictures Animation for Netflix.\nThe film features the voices of Arden Cho, Ahn Hyo-seop, May Hong, Ji-young Yoo, Yunjin Kim, Daniel Dae Kim, Ken Jeong, and Lee Byung-hun.\nIt tells the story of a K-pop girl group called Huntr/x,[a] who secretly work as demon hunters.\nThey face off against a rival boy band, the Saja Boys, whose members are secretly demons.\nKang was inspired to create the story by her Korean heritage. She combined elements of mythology, demon stories, and K-pop to make a film that is both visually unique and culturally meaningful.\nBy March 2021, the project was officially in production at Sony Pictures Animation, with the full creative team on board.\nThe animation was done by Sony Pictures Imagew